In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import re
sys.path.append('../')
from common import *
from tqdm import tqdm
import multiprocessing
from functools import partial
tqdm.pandas()

In [16]:
YMDB_success_met_smile_path = '../../../Results/not_lipid/top50_0.3_re/YMDB_success_met_smile_top50_0.3.pickle'
YMDB_success_met_smile_add_no_ec_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/YMDB_success_met_smile_top50_0.3.pickle'
YMDB_success_met_smile_add_no_ec_uptake_file = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/YMDB_success_met_smile_uptake_top50_0.3.pickle'
YMDB_success_met_smile_uptake_file = '../../../Results/not_lipid/top50_0.3_re/YMDB_success_met_smile_uptake_top50_0.3.pickle'
rxndb_to_model_path = '../../../Results/not_lipid/top50_0.3_re/rxndb_to_model_top50_0.3.csv'
rxndb_to_model_with_target_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_to_model_with_target_top50_0.3.csv'
rxndb_to_model_with_target_success_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_to_model_with_target_top50_0.3_success.csv'
rxndb_to_model_add_no_ec_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_to_model_top50_0.3.csv'
rxndb_met_max_score_pd_top50_0_3_path = '../../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_met_max_score_pd_top50_0.3.csv'  
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
yeast_model_path = '../../../Data/model/yeast-GEM-final.csv'
uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
rxndb_met_max_score_file = '../../../Results/rxndb_met_max_score_pd_top50_0.3_re.csv'

# lipid_YMDB_success_met_smile_file_remian ='../../../Results/lipid/topall_0.3_remain/YMDB_success_met_smile_topall_0.3_remain.pickle'
lipid_YMDB_success_met_smile_file_remian ='../../../Results/lipid/topall_0.3_remain_re/YMDB_success_met_smile_topall_0.3_remain.pickle'
# lipid_YMDB_success_met_smile_file = '../../../Results/lipid/topall_0.3/YMDB_success_met_smile_topall_0.3.pickle'
lipid_YMDB_success_met_smile_file = '../../../Results/lipid/topall_0.3_re/YMDB_success_met_smile_topall_0.3.pickle'

In [17]:
def get_score_from_smiles(input_smiles,smile_max_score_pd): ###return highest similarity score
    # input_smiles = normalize_smiles(input_smiles)
    row = smile_max_score_pd[smile_max_score_pd['inchikey0'] == smiles2inchikey0(input_smiles)]
    if not row.empty:
        return row['score'].max()
    else:
        return 0
def process_reaction_product(index_row,smile_max_score_pd):
    index, row = index_row
    reactant_smiles = row['rxn_smiles_basic'].split('>>')[0].split('.')
    product_smiles = row['rxn_smiles_basic'].split('>>')[1].split('.')
    scores = []

    for i in product_smiles:
        scores.append(get_score_from_smiles(i, smile_max_score_pd))

    scores_all_0 = all(score == 1 for score in scores)

    if scores_all_0 and len(reactant_smiles)>0:
        return reactant_smiles, row['NO']
    else:
        return [], None
def process_reaction_reactant(index_row,smile_max_score_pd):
    index, row = index_row
    reactant_smiles = row['rxn_smiles_basic'].split('>>')[0].split('.')
    product_smiles = row['rxn_smiles_basic'].split('>>')[1].split('.')
    scores = []

    for i in reactant_smiles:
        scores.append(get_score_from_smiles(i, smile_max_score_pd))

    scores_all_0 = all(score == 1 for score in scores)

    if scores_all_0 and len(product_smiles)>0:
        return product_smiles, row['NO']
    else:
        return [], None
def process_chunk_reactant(chunk_df,smile_max_score_pd):
    smiles_success = []
    success_rxndbid = []
    for result in map(partial(process_reaction_reactant,smile_max_score_pd=smile_max_score_pd), chunk_df.iterrows()):
        if result[0]:  # check whether the result is non-empty
            smiles_success.extend(result[0])
        if result[1] is not None:
            success_rxndbid.append(result[1])
    return smiles_success, success_rxndbid
def process_chunk_product(chunk_df,smile_max_score_pd):
    smiles_success = []
    success_rxndbid = []
    for result in map(partial(process_reaction_product,smile_max_score_pd=smile_max_score_pd), chunk_df.iterrows()):
        if result[0]:  # check whether the result is non-empty
            smiles_success.extend(result[0])
        if result[1] is not None:
            success_rxndbid.append(result[1])
    return smiles_success, success_rxndbid
def process_reactions_in_parallel_reactant(rxndb,origin_smile_max_score_pd, num_processes=100, num_iterations=1):
    num = 0
    tmp_smile_max_score_pd = origin_smile_max_score_pd
    while num < num_iterations:
        num+=1
        # pool = multiprocessing.Pool(num_processes)
        smiles_success = []
        success_rxndbid = []
        chunks = np.array_split(rxndb, 100)
        with multiprocessing.Pool(num_processes) as pool:
            for result in tqdm(pool.imap(partial(process_chunk_reactant,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                smiles_success.extend(result[0])
                success_rxndbid.extend(result[1])
            for result in tqdm(pool.imap(partial(process_chunk_product,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                smiles_success.extend(result[0])
                success_rxndbid.extend(result[1])
        # pool.close()
        # pool.join()
        
        smiles_success = list(set(smiles_success))
        success_rxndbid = list(set(success_rxndbid))
        # Process cumulative successful SMILES
        for smile in smiles_success:
            if get_score_from_smiles(smile, tmp_smile_max_score_pd) < 1:
                new_row = {'smile': smile, 'score': 1, 'sim_smile': 'sys'}
                tmp_smile_max_score_pd = tmp_smile_max_score_pd._append(new_row, ignore_index=True)

        # smiles_success = [normalize_smiles(met) for met in smiles_success]
        # smiles_success = list(set(smiles_success))

        print(f'Iteration {num} - Current success count: {len(smiles_success)}')
        print(f'Iteration {num} - Current success_rxndbid count: {len(success_rxndbid)}')
        print('============================================================================')

    print('final success', len(smiles_success))
    print('final success_rxndbid', len(success_rxndbid))
    # print(success_rxndbid)
    return smiles_success, success_rxndbid,tmp_smile_max_score_pd

In [18]:
yeast8_total_smiles = get_all_smiles_in_model(yeast_model_path,ymdb_output_final)
success_smiles = load_pickle(YMDB_success_met_smile_add_no_ec_uptake_file)
lipid_success_smiles = load_pickle(lipid_YMDB_success_met_smile_file)   
lipid_success_smiles_remain = load_pickle(lipid_YMDB_success_met_smile_file_remian)
uptake_met = pd.read_csv(uptake_met_path)   
uptake_smiles = uptake_met['SMILES'].to_list()
sink_smiles = yeast8_total_smiles + success_smiles + uptake_smiles + lipid_success_smiles + lipid_success_smiles_remain
sink_smiles = list(set(sink_smiles))
sink0 = [smiles2inchikey0(x) for x in sink_smiles]
sink0 = list(set(sink0))
print(len(sink0))


14540


In [19]:
YMDB_success_met_smile = load_pickle(YMDB_success_met_smile_path)
YMDB_success_met_smile_add_no_ec = load_pickle(YMDB_success_met_smile_add_no_ec_path)
differenct_metabolites = [x for x in YMDB_success_met_smile_add_no_ec if x not in YMDB_success_met_smile]
differenct_metabolites

['CCCCCCCCCCCCCC=CC(O)C(N)CO',
 'O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O',
 'N#CC(N)CCC(=O)O',
 'CCCC(N)C(=O)O',
 'CC1=C(O)C(=O)OC1C',
 'O=C(C=Cc1ccc(O)c(O)c1)OC(C(=O)O)C(O)C(=O)O',
 'COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O',
 'CCCC([O-])=S',
 'O=C(O)C(CO)c1ccccc1',
 'Cc1ccc(O)c([N+](=O)[O-])c1']

In [20]:
YMDB_success_met_smile = load_pickle(YMDB_success_met_smile_uptake_file)
YMDB_success_met_smile_add_no_ec = load_pickle(YMDB_success_met_smile_add_no_ec_uptake_file)
differenct_metabolites_ = [x for x in YMDB_success_met_smile_add_no_ec if x not in YMDB_success_met_smile]
print(differenct_metabolites_)
set(differenct_metabolites) & set(differenct_metabolites_)

['CCCCCCCCCCCCCC=CC(O)C(N)CO', 'O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O', 'N#CC(N)CCC(=O)O', 'CCCC(N)C(=O)O', 'CC1=C(O)C(=O)OC1C', 'CC1OC(Oc2cc(O)c3c(=O)c(OC4OC(CO)C(O)C(O)C4O)c(-c4ccc(O)cc4)oc3c2)C(O)C(O)C1O', 'CCCC([O-])=S', 'O=C(O)C(CO)c1ccccc1', 'Cc1ccc(O)c([N+](=O)[O-])c1']


{'CC1=C(O)C(=O)OC1C',
 'CCCC(N)C(=O)O',
 'CCCC([O-])=S',
 'CCCCCCCCCCCCCC=CC(O)C(N)CO',
 'Cc1ccc(O)c([N+](=O)[O-])c1',
 'N#CC(N)CCC(=O)O',
 'O=C(O)C(CO)c1ccccc1',
 'O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O'}

In [21]:
differenct_metabolites_all =set(differenct_metabolites) | set(differenct_metabolites_)


In [23]:
differenct_metabolites_inchikey0 = [smiles2inchikey0(x) for x in differenct_metabolites_all]
rxndb_to_model_add_no_ec = pd.read_csv(rxndb_to_model_add_no_ec_path)
rxndb_to_model_add_no_ec.head()

,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
0,rxn1,NaN,MNXR164590,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...
1,rxn2,NaN,MNXR164590,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...
2,rxn3,3.2.1.74,MNXR173469_reverse,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...
3,rxn4,3.2.1.74,MNXR173469_reverse,O.OCC1O[C@@H](OC2C(CO)OC(O)C(O)C2O)C(O)C(O)C1O...,O.OCC1O[C@@H](OC2C(CO)OC(O)C(O)C2O)C(O)C(O)C1O...
4,rxn5,3.2.1.74,MNXR173469_reverse,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...


In [24]:
def find_reaction_containing_metabolite(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['product_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[1].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['product_inchikey0'].apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]
def find_reaction_containing_metabolite_(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['reactant_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[0].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['reactant_inchikey0'].apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]

In [10]:
rxndb_to_model_add_no_ec.head()

,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
0,rxn1,NaN,MNXR164590,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...,CC(C)=CCCC(C)C1CCC2C3=C(CCC21C)C1(C)CCC(O)C(C)...
1,rxn2,NaN,MNXR164590,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...,CC(C=O)CCCC(C)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC...
2,rxn3,3.2.1.74,MNXR173469_reverse,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...
3,rxn4,3.2.1.74,MNXR173469_reverse,O.OCC1O[C@@H](OC2C(CO)OC(O)C(O)C2O)C(O)C(O)C1O...,O.OCC1O[C@@H](OC2C(CO)OC(O)C(O)C2O)C(O)C(O)C1O...
4,rxn5,3.2.1.74,MNXR173469_reverse,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...,O.OCC1OC(OC2C(CO)O[C@@H](OC3C(CO)OC(O)C(O)C3O)...


In [11]:
rxndb_to_model_add_no_ec_with_target = find_reaction_containing_metabolite(differenct_metabolites_inchikey0, rxndb_to_model_add_no_ec)
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target.drop(['product_inchikey0','target_num'],axis=1)

In [12]:
rxndb_to_model_add_no_ec_with_target_ = find_reaction_containing_metabolite_(differenct_metabolites_inchikey0, rxndb_to_model_add_no_ec)
rxndb_to_model_add_no_ec_with_target_ = rxndb_to_model_add_no_ec_with_target_.drop(['product_inchikey0','reactant_inchikey0','target_num'],axis=1)


In [13]:
rxndb_to_model_add_no_ec_with_target = pd.concat([rxndb_to_model_add_no_ec_with_target, rxndb_to_model_add_no_ec_with_target_])
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target.drop_duplicates(subset=['NO'])
rxndb_to_model_add_no_ec_with_target.shape

(446, 5)

In [14]:
rxndb_to_model_add_no_ec_with_target.to_csv(rxndb_to_model_with_target_path, index=False)


In [25]:
rxndb_to_model_add_no_ec_with_target = pd.read_csv(rxndb_to_model_with_target_path)

In [26]:
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target[rxndb_to_model_add_no_ec_with_target['EC number'].isna()]
rxndb_to_model_add_no_ec_with_target.shape

(73, 5)

In [27]:
rxndb_met_max_score_pd = pd.DataFrame({'smile': sink_smiles})
rxndb_met_max_score_pd['score'] = 1
rxndb_met_max_score_pd['sim_smile'] = ''
rxndb_met_max_score_pd['inchikey0'] = rxndb_met_max_score_pd['smile'].apply(smiles2inchikey0)
rxndb_met_max_score_pd.to_csv(rxndb_met_max_score_file, index=False)

In [10]:
rxndb_met_max_score_pd = pd.read_csv(rxndb_met_max_score_file)


In [27]:
# rxndb_met_max_score_pd_top50_0_3 = pd.read_csv(rxndb_met_max_score_pd_top50_0_3_path)
# rxndb_met_max_score_pd_top50_0_3['inchikey0'] = rxndb_met_max_score_pd_top50_0_3['smile'].apply(smiles2inchikey0)
# rxndb_met_max_score_pd = pd.concat([rxndb_met_max_score_pd,rxndb_met_max_score_pd_top50_0_3])

In [28]:
# rxndb_met_max_score_pd_top50_0_3  = pd.read_csv(rxndb_met_max_score_pd_top50_0_3_path)
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_to_model_add_no_ec_with_target, rxndb_met_max_score_pd, num_processes=5, num_iterations=2)


 21%|██        | 21/100 [00:00<00:00, 198.55it/s]

100%|██████████| 100/100 [00:00<00:00, 210.29it/s]


Iteration 1 - Current success count: 45
Iteration 1 - Current success_rxndbid count: 73


100%|██████████| 100/100 [00:00<00:00, 209.76it/s]


Iteration 2 - Current success count: 45
Iteration 2 - Current success_rxndbid count: 73
final success 45
final success_rxndbid 73


In [29]:
rxndb_to_model_success = rxndb_to_model_add_no_ec_with_target[rxndb_to_model_add_no_ec_with_target['NO'].isin(success_rxndbid)]
rxndb_to_model_success.shape

(73, 5)

In [32]:
rxndb_to_model_success.to_csv(rxndb_to_model_with_target_success_path, index=False)

In [31]:
rxndb_to_model_success.head()

,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
0,rxn801,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
1,rxn808,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
2,rxn815,2.1.1.M6,MNXR120324_reverse,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...,COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O.Nc1n...
3,rxn1290,3.1.3;3.1.3.-,MNXR116844,CCCCCCCCCCCCCC=CC(O)C([NH3+])COP(=O)([O-])[O-]...,CCCCCCCCCCCCCC=CC(O)C([NH3+])COP(=O)([O-])[O-]...
4,rxn1297,3.1.3;3.1.3.-,MNXR116844,CCCCCCCCCCCCCC=CC(O)C(N)COP(=O)([O-])[O-].O>>C...,CCCCCCCCCCCCCC=CC(O)C(N)COP(=O)([O-])[O-].O>>C...


In [30]:
success_inchikey0 = [smiles2inchikey0(x) for x in smiles_success]
for i in differenct_metabolites_inchikey0:
    if i not in success_inchikey0:
        print(i)

DGAODIKUWGRDBO
SNDPXSYFESPGGJ


In [31]:
len(differenct_metabolites_inchikey0)

11